In [2]:
## Install packages
!uv pip install pydantic
!uv pip install instructor

Audited 1 package in 2ms
Audited 1 package in 289ms


In [4]:
import getpass
import os
import pathlib
from io import BytesIO

import instructor
import requests
from openai import OpenAI
from pydantic import BaseModel

In [5]:
_API_KEY = getpass.getpass("Your API key: \n")

Your API key: 
 ········


In [87]:
client = instructor.from_provider(
    "openai/gpt-4o-mini",
    mode=instructor.Mode.RESPONSES_TOOLS,
    api_key=_API_KEY,
)

## [File search](https://developers.openai.com/api/docs/guides/tools-file-search) tool

[File search tool in OpenAI](https://developers.openai.com/api/docs/guides/tools-file-search) allows [semantic search](https://developers.openai.com/api/docs/guides/retrieval#semantic-search) and retrieval of information from a knowledge base. This will require creating a knowledge base (a `vector store`). A `vector store` can contain multiple files, which will be processed to allow search and retrieval.

In the following sections, we will:

1. Prepare files to upload (`create_file` function)
2. Create a vector store
3. Add/upload files to the vector store
4. Conduct search

In [82]:
def create_file(client, file_path):
    # From https://developers.openai.com/api/docs/guides/tools-file-search#upload-the-file-to-the-file-api
    openai_client = client.client

    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = openai_client.files.create(file=file_tuple, purpose="assistants")
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = openai_client.files.create(file=file_content, purpose="assistants")
    return result.id

In [83]:
# Replace with your own file path or URL

# Local file
# Downloaded from https://budgetandfinance.psu.edu/sites/budgetandfinance/files/right_to_know_2024.pdf
# Then, uploaded to Jupyter

right_to_know_file_id = create_file(client, "right_to_know_2024.pdf")
print("right_to_know_file_id: " + right_to_know_file_id)

# Online file
audit_file_id = create_file(client, "https://budgetandfinance.psu.edu/sites/budgetandfinance/files/psu-fy25-audited-financial-statements-final.pdf")
print("audit_file_id: " + audit_file_id)


right_to_know_file_id: file-P1VQiFEwUwTnsTHTH1wpPr
audit_file_id: file-MFDm1hSNyuJMKdL6BbXqWJ


In [56]:
def create_vector_store(client, name):
    # From https://developers.openai.com/api/docs/guides/tools-file-search#create-a-vector-store
    openai_client = client.client

    return openai_client.vector_stores.create(name=name, expires_after={"days": 30, "anchor": "last_active_at"})

In [57]:
vector_store = create_vector_store(client, "ist-597-activity-04")

In [99]:
print(vector_store.id)

vs_698e5694b43c8191a064f99e40ead5f7


In [48]:
def add_file_to_vector_store(client, vector_store, file_id):
    openai_client = client.client

    r = openai_client.vector_stores.files.create(
        vector_store_id=vector_store.id, file_id=file_id
    )

    return r

In [84]:
added_right_to_know = add_file_to_vector_store(client, vector_store, right_to_know_file_id)
added_audited = add_file_to_vector_store(client, vector_store, audit_file_id)

In [60]:
print(added_right_to_know)

VectorStoreFile(id='file-6tWc3MidiBAjB9icKUHjKp', created_at=1770935961, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_698e5694b43c8191a064f99e40ead5f7', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))


In [110]:
# Let's check status
result = client.client.vector_stores.files.list(
    vector_store_id=vector_store.id
)
print(result)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-P1VQiFEwUwTnsTHTH1wpPr', created_at=1770936704, last_error=None, object='vector_store.file', status='completed', usage_bytes=520472, vector_store_id='vs_698e5694b43c8191a064f99e40ead5f7', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static')), VectorStoreFile(id='file-MFDm1hSNyuJMKdL6BbXqWJ', created_at=1770936705, last_error=None, object='vector_store.file', status='completed', usage_bytes=243872, vector_store_id='vs_698e5694b43c8191a064f99e40ead5f7', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-P1VQiFEwUwTnsTHTH1wpPr', last_id='file-MFDm1hSNyuJMKdL6BbXqWJ')


## Let's check the vector store

* Go to <https://platform.openai.com/storage/vector_stores>
* Make sure to select the correct project on top
* Do you see the vector store and the files?
  * Might take some time for the files to process and show up

In [135]:
def search_vector_store(query, vector_store_id, max_num_results=3):
    try:
        results_page = client.vector_stores.search(
            vector_store_id=vector_store_id,
            query=query,
            max_num_results=max_num_results
        )
    
        # Process the results (results_page is a Page object containing SearchResult objects in its 'data' attribute)
        for data in results_page.data:
            print("-" * 30)
            print(f"file_id: {data.file_id}, filename: {data.filename}")
            print(f"score: {data.score}")
            # Extract content (content is a list of content objects)
            content_text = ''.join(content.text for content in data.content)
            print(f"content: {content_text}") # Print first 200 chars

            return results_page
    
    except Exception as e:
        print(f"An error occurred: {e}")

In [136]:
q = "What is Penn State's operating revenues?"
r = search_vector_store(q, vector_store.id)

------------------------------
file_id: file-MFDm1hSNyuJMKdL6BbXqWJ, filename: psu-fy25-audited-financial-statements-final.pdf
score: 0.8602772999117246
content: 4 THE PENNSYLVANIA STATE UNIVERSITY
Financial Overview
The following section provides summarized results of the financial performance and position of the Pennsylvania 
State University (“Penn State”, or the “University”), and as a result it should be read alongside the accompanying 
consolidated financial statements and notes to the financial statements. All figures in this section are consolidated 
and – unless specifically noted – include the University, Penn State Health, and other subsidiaries (see note one to the 
financial statements).
OPERATING RESULTS
Penn State’s net assets increased by $1.06 billion during the fiscal year ended June 30, 2025 (FY2025), a result of 
strong operating results at both the University and at Penn State Health alongside positive non-operating activities, 
particularly in the form of unrealiz

In [138]:
q = "How much did Penn State spend University funds on research?"
r = search_vector_store(q, vector_store.id)

------------------------------
file_id: file-MFDm1hSNyuJMKdL6BbXqWJ, filename: psu-fy25-audited-financial-statements-final.pdf
score: 0.9696414377347563
content: AUDITED FINANCIAL STATEMENTS FY2025 3
Strategic investments and initiatives
• Building on the success of the new budget allocation model introduced in FY2024, the 
University has continued to fund strategic priorities effectively.
• Penn State invested $322 million of University funds on research in FY2025, an increase 
of $22 million on the record-breaking spend over the previous fiscal year.
• The Office of Investment Management, under the oversight of the Penn State Investment 
Council, continued its strong performance. The 10-year annualized return as of June 30, 
2025, for the long-term investment pool was 8.80%.
• Targeted investments were made to attract and retain top faculty talent, including seed 
grants, retention packages, and professional development opportunities.
• FY2025 was a record-breaking year for fundraisi

## What if we feed this response to OpenAI to extract specific answer?

That is, we can have two step process:

1. Extract relevant text from all given sources
2. Perform the user query in returned text (i.e., similar to our previous class activity)